In [1]:
import matplotlib.pyplot as plt
import os
import pandas as pd

In [2]:
output_dir = "graficos_adosamiento_edificio"
os.makedirs(output_dir, exist_ok=True)

# Se crea un directorio aparte para ir almacanando los plots por distrito

In [3]:
nepal = pd.read_csv("C:\\Users\\inilo\\The_Bridge\\Iñigo\\EDA_Iñigo_Lopez\\data\\eq2015.csv")
fuente_1 = "https://www.kaggle.com/datasets/sanskarnegi/nepal-earthquake-2015"

In [ ]:
distritos = nepal["district_name"].unique()
distritos

# Se crea una variable que almacena los nombres de los distritos afectados por terremotos

array(['Okhaldhunga', 'Sindhuli', 'Ramechhap', 'Dolakha', 'Sindhupalchok',
       'Kavre', 'Nuwakot', 'Rasuwa', 'Dhading', 'Makwanpur', 'Gorkha'],
      dtype=object)

In [6]:
graficos_por_distrito = {}

# Se crea un diccionario que almacena los path de cada plot generado

In [7]:
for distrito in distritos: # Se recorren todos los distritos en la lista distritos
    df_distrito = nepal[nepal["district_name"] == distrito] # Se define esta variable como el nombre del distrito
    df_grouped = df_distrito.groupby(["position", "damage_grade"]).size().unstack(fill_value=0) # Se agrupan los valores por tipo de adosamiento (position)
    # y el grado de destrucción
    
    # Con esto nos aseguramos de que todos los grados de destrucción estén en el dataframe agrupado
    todos_los_grados = ["Grade 1", "Grade 2", "Grade 3", "Grade 4", "Grade 5"]
    df_grouped = df_grouped.reindex(columns=todos_los_grados, fill_value=0)
    
    # Misma para poder comprobar los tipos de adosamiento
    orden_adosamiento = ["Not attached", "Attached-1 side", "Attached-2 side", "Attached-3 side"]
    df_grouped = df_grouped.reindex(orden_adosamiento,fill_value=0)

    # Se normalizan los resultados para poder ver los plots a la misma altura a pesar de tener menos cantidad de datos por categoría
    df_normalized = df_grouped.div(df_grouped.sum(axis=1), axis=0).fillna(0)

    # Se crea la figura
    fig, ax = plt.subplots(figsize=(8, 10))
    df_normalized.plot(kind='bar', stacked=True, colormap='viridis', ax=ax,legend=False)

    # Se añaden las etiquetas
    plt.xlabel('Tipo de adosamiento')
    plt.ylabel('Número de edificios')
    plt.title(f'Distrito: {distrito}')
    plt.xticks(rotation=0)
    ax.set_ylim(0, 1)

    # Con estas líneas de código se añaden los porcentajes correspondientes de cada porción de los barplots en %
    for i, (idx, row) in enumerate(df_normalized.iterrows()):
        y_offset = 0
        for damage_grade, value in row.items():
            if value > 0:
                ax.text(
                    i + 0.48,
                    y_offset + value / 2, 
                    f"{value * 100:.1f}%", 
                    ha='right', 
                    va='center',
                    fontsize=7,
                    color='black'
                )
            y_offset += value

    # Se guarda la imagen en memoria
    img_path = os.path.join(output_dir, f"{distrito}.png")
    plt.savefig(img_path, bbox_inches="tight", dpi=150)
    plt.close(fig)

    # Se guarda la ruta de la imagen
    graficos_por_distrito[distrito] = img_path

In [11]:
# Para ir haciendo los tests estadíticos por distrito se aislan los datos de cada distrito y se trabaja con ellos
lista_gorkha = nepal[nepal["district_name"] == "Gorkha"]

In [ ]:
from scipy.stats import chi2_contingency, fisher_exact
# Se importan las librerías para hacer los tests de chi cuadrado y el test exacto de Fisher

# Se crea una función para hacer el análisis entre dos tipos de adosamiento
def analizar_contraste(position_a, position_b):
    print(f"\n Se está comparando: {position_a} y {position_b}")
    
    # Se filtran los datos de los dos grupos de adosamiento que se está analizando
    subset = lista_gorkha[lista_gorkha["position"].isin([position_a, position_b])].copy()

    # 1. Test de Chi-cuadrado
    
    tabla_chi = pd.crosstab(subset["position"], subset["damage_grade"])
    chi2, p_chi2, dof, expected = chi2_contingency(tabla_chi)
    
    print(f"\nTest de Chi-cuadrado:")
    print(f"Chi² = {chi2:.2f}, dof = {dof}, p-valor = {p_chi2:.4e}")
    
    # 2. Test exacto de Fisher (colapsado vs no colapsado)

    subset["colapsado"] = subset["damage_grade"] == "Grade 5"
    tabla_fisher = pd.crosstab(subset["position"], subset["colapsado"])

# Comparaciones deseadas
analizar_contraste("Not attached", "Attached-1 side")
analizar_contraste("Not attached", "Attached-2 side")
analizar_contraste("Not attached", "Attached-3 side")


 se está comparando: Not attached y Attached-1 side

Test de Chi-cuadrado:
Chi² = 30.70, dof = 4, p-valor = 3.5211e-06

 se está comparando: Not attached y Attached-2 side

Test de Chi-cuadrado:
Chi² = 845.80, dof = 4, p-valor = 9.2148e-182

 se está comparando: Not attached y Attached-3 side

Test de Chi-cuadrado:
Chi² = 34.73, dof = 4, p-valor = 5.2887e-07


In [16]:
lista_dolakha = nepal[nepal["district_name"] == "Dolakha"]

In [18]:
# Se crea una función para hacer el análisis entre dos tipos de adosamiento
def analizar_contraste(position_a, position_b):
    print(f"\n Se está comparando: {position_a} y {position_b}")
    
    # Se filtran los datos de los dos grupos de adosamiento que se está analizando
    subset = lista_dolakha[lista_dolakha["position"].isin([position_a, position_b])].copy()

    # 1. Test de Chi-cuadrado
    
    tabla_chi = pd.crosstab(subset["position"], subset["damage_grade"])
    chi2, p_chi2, dof, expected = chi2_contingency(tabla_chi)
    
    print(f"\nTest de Chi-cuadrado:")
    print(f"Chi² = {chi2:.2f}, dof = {dof}, p-valor = {p_chi2:.4e}")
    
    # 2. Test exacto de Fisher (colapsado vs no colapsado)

    subset["colapsado"] = subset["damage_grade"] == "Grade 5"
    tabla_fisher = pd.crosstab(subset["position"], subset["colapsado"])

# Comparaciones deseadas
analizar_contraste("Not attached", "Attached-1 side")
analizar_contraste("Not attached", "Attached-2 side")
analizar_contraste("Not attached", "Attached-3 side")


 Se está comparando: Not attached y Attached-1 side

Test de Chi-cuadrado:
Chi² = 46.94, dof = 4, p-valor = 1.5713e-09

 Se está comparando: Not attached y Attached-2 side

Test de Chi-cuadrado:
Chi² = 656.28, dof = 4, p-valor = 1.0175e-140

 Se está comparando: Not attached y Attached-3 side

Test de Chi-cuadrado:
Chi² = 10.09, dof = 4, p-valor = 3.8965e-02


In [19]:
lista_makwanpur = nepal[nepal["district_name"] == "Makwanpur"]

In [20]:
# Se crea una función para hacer el análisis entre dos tipos de adosamiento
def analizar_contraste(position_a, position_b):
    print(f"\n Se está comparando: {position_a} y {position_b}")
    
    # Se filtran los datos de los dos grupos de adosamiento que se está analizando
    subset = lista_makwanpur[lista_makwanpur["position"].isin([position_a, position_b])].copy()

    # 1. Test de Chi-cuadrado
    
    tabla_chi = pd.crosstab(subset["position"], subset["damage_grade"])
    chi2, p_chi2, dof, expected = chi2_contingency(tabla_chi)
    
    print(f"\nTest de Chi-cuadrado:")
    print(f"Chi² = {chi2:.2f}, dof = {dof}, p-valor = {p_chi2:.4e}")
    
    # 2. Test exacto de Fisher (colapsado vs no colapsado)

    subset["colapsado"] = subset["damage_grade"] == "Grade 5"
    tabla_fisher = pd.crosstab(subset["position"], subset["colapsado"])

# Comparaciones deseadas
analizar_contraste("Not attached", "Attached-1 side")
analizar_contraste("Not attached", "Attached-2 side")
analizar_contraste("Not attached", "Attached-3 side")


 Se está comparando: Not attached y Attached-1 side

Test de Chi-cuadrado:
Chi² = 194.39, dof = 4, p-valor = 6.0271e-41

 Se está comparando: Not attached y Attached-2 side

Test de Chi-cuadrado:
Chi² = 219.30, dof = 4, p-valor = 2.6472e-46

 Se está comparando: Not attached y Attached-3 side

Test de Chi-cuadrado:
Chi² = 30.39, dof = 4, p-valor = 4.0708e-06


In [21]:
lista_rasuwa = nepal[nepal["district_name"] == "Rasuwa"]

In [22]:
# Se crea una función para hacer el análisis entre dos tipos de adosamiento
def analizar_contraste(position_a, position_b):
    print(f"\n Se está comparando: {position_a} y {position_b}")
    
    # Se filtran los datos de los dos grupos de adosamiento que se está analizando
    subset = lista_rasuwa[lista_rasuwa["position"].isin([position_a, position_b])].copy()

    # 1. Test de Chi-cuadrado
    
    tabla_chi = pd.crosstab(subset["position"], subset["damage_grade"])
    chi2, p_chi2, dof, expected = chi2_contingency(tabla_chi)
    
    print(f"\nTest de Chi-cuadrado:")
    print(f"Chi² = {chi2:.2f}, dof = {dof}, p-valor = {p_chi2:.4e}")
    
    # 2. Test exacto de Fisher (colapsado vs no colapsado)

    subset["colapsado"] = subset["damage_grade"] == "Grade 5"
    tabla_fisher = pd.crosstab(subset["position"], subset["colapsado"])

# Comparaciones deseadas
analizar_contraste("Not attached", "Attached-1 side")
analizar_contraste("Not attached", "Attached-2 side")
analizar_contraste("Not attached", "Attached-3 side")


 Se está comparando: Not attached y Attached-1 side

Test de Chi-cuadrado:
Chi² = 81.07, dof = 4, p-valor = 1.0332e-16

 Se está comparando: Not attached y Attached-2 side

Test de Chi-cuadrado:
Chi² = 161.71, dof = 4, p-valor = 6.2808e-34

 Se está comparando: Not attached y Attached-3 side

Test de Chi-cuadrado:
Chi² = 13.74, dof = 4, p-valor = 8.1852e-03
